In [1]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as  plt
import time

In [2]:
df=pd.read_excel("all_data_meituan.xlsx")[["comment","star"]]
df.head()

,comment,star
0,还行吧，建议不要排队那个烤鸭和羊肉串，因为烤肉时间本来就不够，排那个要半小时，然后再回来吃烤...,40
1,去过好几次了 东西还是老样子 没增添什么新花样 环境倒是挺不错 离我们这也挺近 味道还可以 ...,40
2,一个字：好！！！ #羊肉串# #五花肉# #牛舌# #很好吃# #鸡软骨# #拌菜# #抄河...,50
3,第一次来吃，之前看过好多推荐说这个好吃，真的抱了好大希望，排队的人挺多的，想吃得趁早来啊。还...,20
4,羊肉串真的不太好吃，那种说膻不膻说臭不臭的味。烤鸭还行，大虾没少吃，也就到那吃大虾了，吃完了...,30


In [3]:
df.shape

(17400, 2)

In [4]:
df['sentiment']=df['star'].apply(lambda x:1 if x>30 else 0)
df=df.drop_duplicates() ## 去掉重复的评论
df=df.dropna()

In [5]:
X=pd.concat([df[['comment']],df[['comment']],df[['comment']]])
y=pd.concat([df.sentiment,df.sentiment,df.sentiment])
X.columns=['comment']
X.reset_index
X.shape

(3138, 1)

In [6]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
X['cut_comment']=X["comment"].apply(chinese_word_cut)
X['cut_comment'].head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HUANG_~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.150 seconds.
Prefix dict has been built succesfully.


0    还行 吧 ， 建议 不要 排队 那个 烤鸭 和 羊肉串 ， 因为 烤肉 时间 本来 就 不够...
1    去过 好 几次 了   东西 还是 老 样子   没 增添 什么 新花样   环境 倒 是 ...
2    一个 字 ： 好 ！ ！ ！   # 羊肉串 #   # 五花肉 #   # 牛舌 #   ...
3    第一次 来 吃 ， 之前 看过 好多 推荐 说 这个 好吃 ， 真的 抱 了 好 大 希望 ...
4    羊肉串 真的 不太 好吃 ， 那种 说 膻 不 膻 说 臭 不 臭 的 味 。 烤鸭 还 行...
Name: cut_comment, dtype: object

In [7]:
from sklearn.model_selection import  train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,random_state=42,test_size=0.25)

In [8]:
def get_custom_stopwords(stop_words_file):
    with open(stop_words_file,encoding="utf-8") as f:
        custom_stopwords_list=[i.strip() for i in f.readlines()]
    return custom_stopwords_list

In [9]:
stop_words_file = "stopwords.txt"
stopwords = get_custom_stopwords(stop_words_file)
stopwords[-10:]

['100', '01', '02', '03', '04', '05', '06', '07', '08', '09']

In [10]:
from sklearn.feature_extraction.text import  CountVectorizer
vect=CountVectorizer()
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
vect.fit_transform(X_train["cut_comment"])

<2353x1965 sparse matrix of type '<class 'numpy.int64'>'
	with 20491 stored elements in Compressed Sparse Row format>

In [12]:
vect.fit_transform(X_train["cut_comment"]).toarray().shape

(2353, 1965)

In [13]:
# pd.DataFrame(vect.fit_transform(X_train["cut_comment"]).toarray(),columns=vect.get_feature_names()).iloc[:10,:22]
# print(vect.get_feature_names())
# #  数据维数1956，不算很大（未使用停用词）

In [14]:
vect = CountVectorizer(token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',stop_words=frozenset(stopwords)) # 去除停用词
pd.DataFrame(vect.fit_transform(X_train['cut_comment']).toarray(), columns=vect.get_feature_names()).head()
# 1691 columns,去掉以数字为特征值的列，减少了三列编程1691 
# max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
# min_df = 3 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。

,amazing,happy,ktv,pm2,一万个,一个多,一个月,一串,一人,一件,...,麻烦,麻酱,黄喉,黄桃,黄花鱼,黄金,黑乎乎,黑椒,黑胡椒,齐全
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn import  metrics
svc_cl=SVC()
pipe=make_pipeline(vect,svc_cl)
pipe.fit(X_train.cut_comment, y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [16]:
y_pred = pipe.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)

0.63184713375796175

In [17]:
metrics.confusion_matrix(y_test,y_pred)

array([[  0, 289],
       [  0, 496]], dtype=int64)

####  支持向量机分类

In [18]:
from sklearn.svm import SVC
svc_cl=SVC() # 实例化
pipe=make_pipeline(vect,svc_cl)
pipe.fit(X_train.cut_comment, y_train)
y_pred = pipe.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)

0.63184713375796175

#### 支持向量机 网格搜索

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import  Pipeline
# svc=SVC(random_state=1)
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
tfidf=TfidfTransformer()
# ('tfidf',
#                       TfidfTransformer()),
#                      ('clf',
#                       SGDClassifier(max_iter=1000)),
# svc=SGDClassifier(max_iter=1000)
svc=SVC()
# pipe=make_pipeline(vect,SVC)
pipe_svc=Pipeline([("scl",vect),('tfidf',tfidf),("clf",svc)])
para_range=[0.0001,0.001,0.01,0.1,1.0,10,100,1000]
para_grid=[
    {'clf__C':para_range,
    'clf__kernel':['linear']},
    {'clf__gamma':para_range,
    'clf__kernel':['rbf']}
]

In [20]:
gs=GridSearchCV(estimator=pipe_svc,param_grid=para_grid,cv=10,n_jobs=-1)

In [21]:
gs.fit(X_train.cut_comment,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('scl', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'clf__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10, 100, 1000], 'clf__kernel': ['linear']}, {'clf__gamma': [0.0001, 0.001, 0.01, 0.1, 1.0, 10, 100, 1000], 'clf__kernel': ['rbf']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [22]:
gs.best_estimator_.fit(X_train.cut_comment,y_train)

Pipeline(memory=None,
     steps=[('scl', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [23]:
y_pred = gs.best_estimator_.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)

0.95031847133757963

#### 临近法

In [24]:
from sklearn.neighbors import  KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=4,p=2,metric='minkowski')
pipe=make_pipeline(vect,knn)
pipe.fit(X_train.cut_comment, y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=...owski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform'))])

In [25]:
y_pred = pipe.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)

0.81910828025477711

In [26]:
metrics.confusion_matrix(y_test,y_pred)

array([[188, 101],
       [ 41, 455]], dtype=int64)

#### 决策树

In [27]:
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(criterion='entropy',random_state=1)

In [28]:
pipe=make_pipeline(vect,tree)
pipe.fit(X_train.cut_comment, y_train)
y_pred = pipe.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)

0.93885350318471339

In [29]:
metrics.confusion_matrix(y_test,y_pred)

array([[256,  33],
       [ 15, 481]], dtype=int64)

#### 随机森林

In [30]:

from sklearn.ensemble import RandomForestClassifier
forest=RandomForestClassifier(criterion='entropy',random_state=1,n_jobs=2)
pipe=make_pipeline(vect,forest)
pipe.fit(X_train.cut_comment, y_train)
y_pred = pipe.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)
# 加上tfidf反而准确率96.5至95.0，

0.96560509554140128

In [31]:
metrics.confusion_matrix(y_test,y_pred)

array([[265,  24],
       [  3, 493]], dtype=int64)

#### bagging方法

In [32]:

from sklearn.ensemble import BaggingClassifier
bag=BaggingClassifier(base_estimator=tree,
                     n_estimators=10,
                     max_samples=1.0,
                     max_features=1.0,
                     bootstrap=True,
                     bootstrap_features=False,
                     n_jobs=1,random_state=1)
pipe=make_pipeline(vect,tfidf,bag)
pipe.fit(X_train.cut_comment, y_train)
y_pred = pipe.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)  #  没用转化td-idf 93.2%, 加上转化步骤，准确率提升到95.5

0.95541401273885351

In [33]:
metrics.confusion_matrix(y_test,y_pred)

array([[260,  29],
       [  6, 490]], dtype=int64)

####  Gradient Boosting方法

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
grd = GradientBoostingClassifier(learning_rate=0.18,max_depth=10,n_estimators=240,random_state=42,max_features='sqrt',subsample=0.9,
                                min_impurity_decrease=0.01)
                                
print(grd)
# Choosing subsample < 1.0 leads to a reduction of variance and an increase in bias.
# Choosing max_features < n_features leads to a reduction of variance and an increase in bias.降低过拟合，但是可能会增加偏差，降低方差（对应的过拟合）
pipe=make_pipeline(vect,tfidf,grd)
pipe.fit(X_train.cut_comment, y_train)
y_pred = pipe.predict(X_test.cut_comment)
metrics.accuracy_score(y_test,y_pred)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.18, loss='deviance', max_depth=10,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.01, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=240,
              presort='auto', random_state=42, subsample=0.9, verbose=0,
              warm_start=False)


0.96560509554140128

In [35]:
metrics.confusion_matrix(y_test,y_pred)

array([[265,  24],
       [  3, 493]], dtype=int64)

#### xgboost 方法 sklearn类似 api

In [37]:
from xgboost import XGBClassifier   
# sklearn API 类似于导入的从skearn中导入某个算法，然后再进行实例化即可，初始化算法的时候可以修改默认参数
from xgboost import plot_importance
x_train_vect=vect.fit_transform(X_train["cut_comment"])
x_test_vect= vect.transform(X_test["cut_comment"])
clf = XGBClassifier(
silent=1 ,#设置成1则没有运行信息输出，最好是设置为0.是否在运行升级时打印消息。
# #nthread=4,# cpu 线程数 默认最大
learning_rate= 0.20, # 学习率
min_child_weight=0.5, 
# # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
# #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
# #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
gamma=0.1,  # 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守，一般0.1、0.2这样子。
subsample=0.7, # 随机采样训练样本 训练实例的子采样比
max_depth=15,
max_delta_step=0,#最大增量步长，我们允许每个树的权重估计。
colsample_bylevel=0.7, # Subsample ratio of columns for each split, in each level.
colsample_bytree=0.6, # 生成树时进行的列采样 
reg_lambda=0.0001,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越保守
reg_alpha=0.003, # L1 正则项参数，参数越大，模型越保守
### 正则化是在梯度提升树种没有的，这是xgboost与GB方法的区别之一。
scale_pos_weight=1, #如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。平衡正负权重=sum(负类样本)/sum(正类样本)
# objective= 'reg:logistic', #多分类的问题 指定学习任务和相应的学习目标
objective='binary:logistic' ,
# #num_class=10, # 类别数，多分类与 multisoftmax 并用
n_estimators=800, #树的个数
random_state=42
# #eval_metric= 'auc'
)
evals  = [(x_test_vect, y_test)]
clf.fit(x_train_vect,y_train,eval_set=evals,early_stopping_rounds=150)


[0]	validation_0-error:0.295541
Will train until validation_0-error hasn't improved in 150 rounds.
[1]	validation_0-error:0.277707
[2]	validation_0-error:0.267516
[3]	validation_0-error:0.247134
[4]	validation_0-error:0.242038
[5]	validation_0-error:0.234395
[6]	validation_0-error:0.230573
[7]	validation_0-error:0.225478
[8]	validation_0-error:0.228025
[9]	validation_0-error:0.236943
[10]	validation_0-error:0.238217
[11]	validation_0-error:0.231847
[12]	validation_0-error:0.230573
[13]	validation_0-error:0.224204
[14]	validation_0-error:0.219108
[15]	validation_0-error:0.217834
[16]	validation_0-error:0.215287
[17]	validation_0-error:0.208917
[18]	validation_0-error:0.207643
[19]	validation_0-error:0.2
[20]	validation_0-error:0.201274
[21]	validation_0-error:0.205096
[22]	validation_0-error:0.203822
[23]	validation_0-error:0.198726
[24]	validation_0-error:0.194904
[25]	validation_0-error:0.191083
[26]	validation_0-error:0.189809
[27]	validation_0-error:0.193631
[28]	validation_0-error:

[243]	validation_0-error:0.073885
[244]	validation_0-error:0.076433
[245]	validation_0-error:0.076433
[246]	validation_0-error:0.080255
[247]	validation_0-error:0.077707
[248]	validation_0-error:0.080255
[249]	validation_0-error:0.075159
[250]	validation_0-error:0.072611
[251]	validation_0-error:0.072611
[252]	validation_0-error:0.072611
[253]	validation_0-error:0.072611
[254]	validation_0-error:0.072611
[255]	validation_0-error:0.072611
[256]	validation_0-error:0.072611
[257]	validation_0-error:0.075159
[258]	validation_0-error:0.075159
[259]	validation_0-error:0.075159
[260]	validation_0-error:0.071338
[261]	validation_0-error:0.071338
[262]	validation_0-error:0.071338
[263]	validation_0-error:0.071338
[264]	validation_0-error:0.071338
[265]	validation_0-error:0.071338
[266]	validation_0-error:0.071338
[267]	validation_0-error:0.071338
[268]	validation_0-error:0.071338
[269]	validation_0-error:0.071338
[270]	validation_0-error:0.071338
[271]	validation_0-error:0.071338
[272]	validati

[487]	validation_0-error:0.057325
[488]	validation_0-error:0.057325
[489]	validation_0-error:0.057325
[490]	validation_0-error:0.057325
[491]	validation_0-error:0.059873
[492]	validation_0-error:0.059873
[493]	validation_0-error:0.059873
[494]	validation_0-error:0.059873
[495]	validation_0-error:0.059873
[496]	validation_0-error:0.059873
[497]	validation_0-error:0.059873
[498]	validation_0-error:0.056051
[499]	validation_0-error:0.058599
[500]	validation_0-error:0.058599
[501]	validation_0-error:0.057325
[502]	validation_0-error:0.057325
[503]	validation_0-error:0.057325
[504]	validation_0-error:0.057325
[505]	validation_0-error:0.059873
[506]	validation_0-error:0.059873
[507]	validation_0-error:0.059873
[508]	validation_0-error:0.059873
[509]	validation_0-error:0.059873
[510]	validation_0-error:0.059873
[511]	validation_0-error:0.057325
[512]	validation_0-error:0.057325
[513]	validation_0-error:0.057325
[514]	validation_0-error:0.059873
[515]	validation_0-error:0.059873
[516]	validati

[728]	validation_0-error:0.052229
[729]	validation_0-error:0.052229
[730]	validation_0-error:0.052229
[731]	validation_0-error:0.052229
[732]	validation_0-error:0.052229
[733]	validation_0-error:0.052229
[734]	validation_0-error:0.052229
[735]	validation_0-error:0.052229
[736]	validation_0-error:0.052229
[737]	validation_0-error:0.052229
[738]	validation_0-error:0.052229
[739]	validation_0-error:0.052229
[740]	validation_0-error:0.052229
[741]	validation_0-error:0.052229
[742]	validation_0-error:0.052229
[743]	validation_0-error:0.052229
[744]	validation_0-error:0.052229
[745]	validation_0-error:0.052229
[746]	validation_0-error:0.052229
[747]	validation_0-error:0.052229
[748]	validation_0-error:0.052229
[749]	validation_0-error:0.052229
[750]	validation_0-error:0.052229
[751]	validation_0-error:0.052229
[752]	validation_0-error:0.052229
[753]	validation_0-error:0.052229
[754]	validation_0-error:0.052229
[755]	validation_0-error:0.052229
[756]	validation_0-error:0.052229
[757]	validati

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.7,
       colsample_bytree=0.6, gamma=0.1, learning_rate=0.2,
       max_delta_step=0, max_depth=15, min_child_weight=0.5, missing=None,
       n_estimators=800, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=42, reg_alpha=0.003,
       reg_lambda=0.0001, scale_pos_weight=1, seed=None, silent=1,
       subsample=0.7)

In [38]:
y_pred=clf.predict(x_test_vect)
metrics.accuracy_score(y_test,y_pred)

0.94777070063694269

In [39]:
metrics.confusion_matrix(y_test,y_pred)

array([[257,  32],
       [  9, 487]], dtype=int64)

#### Xgboost learning  API

In [40]:
x_train_vect=vect.fit_transform(X_train["cut_comment"])
x_test_vect= vect.transform(X_test["cut_comment"])
import xgboost as xgb
dtrain = xgb.DMatrix(x_train_vect, label=y_train)
dtest=xgb.DMatrix(x_test_vect,label=y_test)
evals  = [(dtest, 'eval')]
param = {'max_depth':15, 'eta':0.2, 'silent':1, 'objective':'binary:logistic','n_estimators':800, 'gamma':0.1,'colsample_bytree':0.6,
        'alpha':0.003,'lambda':0.04, 'colsample_bylevel':0.7,'subsample':0.7,'min_child_weight':0.5,'evals':evals,'early_stopping_rounds':150 }

bst = xgb.train(param, dtrain,1500)

In [41]:
# print(bst.best_ntree_limit)
y_pred = bst.predict(dtest,ntree_limit=bst.best_ntree_limit)
y_pred=[1 if i>0.5 else 0 for i in y_pred]
# predictions = [round(value) for value in y_pred]  

In [42]:
metrics.accuracy_score(y_test,y_pred)

0.95541401273885351

In [43]:
metrics.confusion_matrix(y_test,y_pred)

array([[257,  32],
       [  3, 493]], dtype=int64)